# Analítica Avanzada de Datos.
---

# Clasificación multiclase

La clasificación multiclase puede considerarse como una combinación de varios clasificadores binarios. Hay dos formas de enfocar el problema:

- **One vs Rest (OVR)**, en la que se crea un clasificador para cada posible valor de clase, con un resultado positivo para los casos en los que la predicción es esta clase, y predicciones negativas para los casos en los que la predicción es cualquier otra clase. Un problema de clasificación con cuatro clases de formas posibles (cuadrado, círculo, triángulo, hexágono) requeriría cuatro clasificadores que predijeran:
    - *cuadrado* o no
    - *círculo* o no
    - *triángulo* o no
    - *hexágono* o no

    
- **One vs One (OVO)**, en el que se crea un clasificador para cada par posible de clases. El problema de clasificación con cuatro clases de formas requeriría los siguientes clasificadores binarios:

    - *cuadrado* o *círculo*
    - *cuadrado* o *triángulo*
    - *cuadrado* o *hexágono*
    - *círculo* o *triángulo*
    - *círculo* o *hexágono*
    - *triángulo* o *hexágono*
    
En ambos enfoques, el modelo global que combina los clasificadores genera un vector de predicciones en el que las probabilidades generadas a partir de los clasificadores binarios individuales se utilizan para determinar qué clase predecir.

> **Más Información**: Para obtener más información de los estimadores con la clasificación multiclase en Scikit-Learn, consulta el documento [Scikit-Learn documentation](https://scikit-learn.org/stable/modules/multiclass.html).

### Explorar los datos¶

Empecemos examinando un conjunto de datos que contenga observaciones de varias clases. Utilizaremos un conjunto de datos que contiene observaciones de tres especies diferentes de pingüinos.

> **Cita**: El dataset de pingüinos utilizado en este ejercicio es un subconjunto de datos recogidos y puestos a disposición por la Dra. Kristen Gorman y la Estación Palmer, Antártida LTER, miembro de la Red de Investigación Ecológica a Largo Plazo.

In [ ]:
import pandas as pd

# cargar el conjunto de datos de entrenamiento
penguins = pd.read_csv('./Dataset/penguins.csv')

# Mostrar una muestra aleatoria de 10 observaciones
sample = penguins.sample(10)
sample

El dataset contiene las siguientes columnas

* **CulmenLength**:  La longitud en mm del culmen (pico) del pingüino.
* **CulmenDepth**: La profundidad en mm del culmen del pingüino.
* **FlipperLength**: Longitud en mm de la aleta del pingüino.
* **BodyMass**: La masa corporal del pingüino en gramos.
* **Species**: Un valor entero que representa la especie del pingüino.

La columna **Species**  es la clase para la que queremos entrenar un modelo de predicción. El dataset incluye tres especies posibles, que se codifican como 0, 1 y 2. Los nombres reales de las especies se revelan mediante el código siguiente:

In [ ]:
penguin_classes = ['Adelie', 'Gentoo', 'Chinstrap']
print(sample.columns[0:5].values, 'SpeciesName')
for index, row in penguins.sample(10).iterrows():
    print('[',row[0], row[1], row[2], row[3], int(row[4]),']',penguin_classes[int(row[4])])

Ahora que ya sabemos qué representan las características y las clases de los datos, vamos a explorar el dataset. En primer lugar, veamos si faltan valores (*null*)

In [ ]:
# Contar el número de valores nulos de cada columna
penguins.isnull().sum()

Parece que faltan algunos valores de características, pero no faltan clases. Profundicemos un poco más y veamos las filas que contienen nulos.

In [ ]:
# Mostrar filas que contengan nulos
penguins[penguins.isnull().any(axis=1)]

Hay dos filas que no contienen ningún valor de característica (*NaN* significa "not a number", es decir, "no es un número"), por lo que no serán útiles para entrenar un modelo. Descartémoslas del dataset.

In [ ]:
# Eliminar filas que contengan valores NaN
penguins=penguins.dropna()
#Confirmar que ahora no hay nulos
penguins.isnull().sum()


Ahora que ya nos hemos ocupado de los valores que faltan, vamos a explorar cómo se relacionan las características con la etiqueta creando algunos gráficos de caja.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

penguin_features = ['CulmenLength','CulmenDepth','FlipperLength','BodyMass']
penguin_label = 'Species'
for col in penguin_features:
    penguins.boxplot(column=col, by=penguin_label, figsize=(6,6))
    plt.title(col)
plt.show()

A partir de los diagramas de caja, parece que las especies 0 y 2 (Adelie y Chinstrap) tienen perfiles de datos similares para la profundidad del culmen, la longitud de las aletas y la masa corporal, pero los Chinstraps tienden a tener culmos más largos. La especie 1 (Gentoo) tiende a tener características claramente diferenciadas de las demás, lo que debería ayudarnos a entrenar un buen modelo de clasificación.

### Preparar los datos

Al igual que para la clasificación binaria, antes de entrenar el modelo, tenemos que separar las características y la clase, y luego dividir los datos en subconjuntos para el entrenamiento y la validación. También aplicaremos una técnica de *estratificación* al dividir los datos para mantener la proporción de cada valor de clase en los datasets de entrenamiento y validación.

In [ ]:
from sklearn.model_selection import train_test_split

# Características y clases separadas
penguins_X, penguins_y = penguins[penguin_features].values, penguins[penguin_label].values

# Dividir los datos 70%-30% en conjunto de entrenamiento y conjunto de prueba.
x_penguin_train, x_penguin_test, y_penguin_train, y_penguin_test = train_test_split(penguins_X, penguins_y,
                                                                                    test_size=0.30,
                                                                                    random_state=0,
                                                                                    stratify=penguins_y)

print ('Training Set: %d, Test Set: %d \n' % (x_penguin_train.shape[0], x_penguin_test.shape[0]))

### Entrenar y evaluar un clasificador multiclase

Ahora que tenemos un conjunto de características de entrenamiento y las etiquetas de entrenamiento correspondientes, podemos ajustar un algoritmo de clasificación multiclase a los datos para crear un modelo. La mayoría de los algoritmos de clasificación de scikit-learn soportan intrínsecamente la clasificación multiclase. Probaremos con un algoritmo de regresión logística.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Establecer la tasa de regularización
reg = 0.1

# entrenar un modelo de regresión logística en el conjunto de entrenamiento
multi_model = LogisticRegression(C=1/reg, solver='lbfgs', multi_class='auto', max_iter=10000).fit(x_penguin_train, y_penguin_train)
print (multi_model)

Ahora podemos utilizar el modelo entrenado para predecir las etiquetas de las características de prueba y comparar las clases predichas con las clases reales:

In [ ]:
penguin_predictions = multi_model.predict(x_penguin_test)
print('Predicted labels: ', penguin_predictions[:15])
print('Actual labels   : ' ,y_penguin_test[:15])

Veamos un informe de clasificación.

In [ ]:
from sklearn. metrics import classification_report

print(classification_report(y_penguin_test, penguin_predictions))

Al igual que con la clasificación binaria, el informe incluye métricas de *precisión* y *recall* para cada clase. Sin embargo, mientras que con la clasificación binaria podríamos centrarnos en las puntuaciones de la clase positiva; en este caso, hay múltiples clases, por lo que tenemos que mirar una métrica global (ya sea la macro o la media ponderada) para tener una idea de lo bien que el modelo se desempeña en las tres clases.

Puede obtener las métricas globales por separado del informe utilizando las clases de puntuación de métricas de scikit-learn, pero con los resultados multiclase debe especificar qué métrica media desea utilizar para la precisión y recall.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print("Overall Accuracy:",accuracy_score(y_penguin_test, penguin_predictions))
print("Overall Precision:",precision_score(y_penguin_test, penguin_predictions, average='macro'))
print("Overall Recall:",recall_score(y_penguin_test, penguin_predictions, average='macro'))

Veamos ahora la matriz de confusión de nuestro modelo:

In [ ]:
from sklearn.metrics import confusion_matrix

# Imprimir la matriz de confusión
mcm = confusion_matrix(y_penguin_test, penguin_predictions)
print(mcm)

La matriz de confusión muestra la intersección entre los valores predichos y los valores reales de las clases para cada clase: en términos sencillos, las intersecciones diagonales de arriba a la izquierda indican el número de predicciones correctas.

Cuando se trata de múltiples clases, suele ser más intuitivo visualizar esto como un mapa de calor, como éste:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(mcm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(penguin_classes))
plt.xticks(tick_marks, penguin_classes, rotation=45)
plt.yticks(tick_marks, penguin_classes)
plt.xlabel("Predicted Species")
plt.ylabel("Actual Species")
plt.show()

Los cuadrados más oscuros en el gráfico de la matriz de confusión indican un gran número de casos, y es de esperar que pueda ver una línea diagonal de cuadrados más oscuros que indican los casos en los que la clase predicha y la real coinciden.

En el caso de un modelo de clasificación multiclase, no es posible una curva ROC única que muestre la tasa de verdaderos positivos frente a la tasa de falsos positivos. Sin embargo, puede utilizar las tasas de cada clase en una comparación Uno contra Resto (OVR) para crear un gráfico ROC para cada clase.

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Obtener puntuaciones de probabilidad de clase
penguin_prob = multi_model.predict_proba(x_penguin_test)

# Obtener métricas ROC para cada clase
fpr = {}
tpr = {}
thresh ={}
for i in range(len(penguin_classes)):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_penguin_test, penguin_prob[:,i], pos_label=i)
    
# Trazar el gráfico ROC
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label=penguin_classes[0] + ' vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label=penguin_classes[1] + ' vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label=penguin_classes[2] + ' vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.show()

Para cuantificar el rendimiento ROC, puede calcular una puntuación agregada del área bajo la curva que se promedia en todas las curvas OVR.

In [ ]:
auc = roc_auc_score(y_penguin_test,penguin_prob, multi_class='ovr')
print('Average AUC:', auc)

### Preprocesamiento de datos en un pipeline

De nuevo, al igual que con la clasificación binaria, puede utilizar un pipeline para aplicar pasos de preprocesamiento a los datos antes de ajustarlos a un algoritmo para entrenar un modelo. Veamos si podemos mejorar el predictor pingüino escalando las características numéricas en un paso de transformación antes del entrenamiento. También probaremos un algoritmo diferente (una máquina de soporte vectorial)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

# Definir preprocesamiento para columnas numéricas (escalarlas)
feature_columns = [0,1,2,3]
feature_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
    ])

# Crear pasos de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('preprocess', feature_transformer, feature_columns)])

# Crear pipeline de entrenamiento
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', SVC(probability=True))])


# ajustar el pipeline para entrenar un modelo de regresión lineal en el conjunto de entrenamiento
multi_model = pipeline.fit(x_penguin_train, y_penguin_train)
print (multi_model)

Ahora podemos evaluar el nuevo modelo.

In [ ]:
# Obtener predicciones a partir de datos de prueba
penguin_predictions = multi_model.predict(x_penguin_test)
penguin_prob = multi_model.predict_proba(x_penguin_test)

# Métricas generales
print("Overall Accuracy:",accuracy_score(y_penguin_test, penguin_predictions))
print("Overall Precision:",precision_score(y_penguin_test, penguin_predictions, average='macro'))
print("Overall Recall:",recall_score(y_penguin_test, penguin_predictions, average='macro'))
print('Average AUC:', roc_auc_score(y_penguin_test,penguin_prob, multi_class='ovr'))

# Matriz de confusión
plt.imshow(mcm, interpolation="nearest", cmap=plt.cm.Blues)
plt.colorbar()
tick_marks = np.arange(len(penguin_classes))
plt.xticks(tick_marks, penguin_classes, rotation=45)
plt.yticks(tick_marks, penguin_classes)
plt.xlabel("Predicted Species")
plt.ylabel("Actual Species")
plt.show()

### Utilizar el modelo con nuevas observaciones de datos

Ahora vamos a guardar nuestro modelo entrenado para poder volver a utilizarlo más adelante.

In [ ]:
import joblib

# Guardar el modelo como un archivo pickle
filename = 'penguin_model.pkl'
joblib.dump(multi_model, filename)

 Ya tenemos un modelo entrenado. Usémoslo para predecir la clase de una nueva observación de un pingüino:

In [ ]:
# Cargar el modelo desde el archivo
multi_model = joblib.load(filename)

# El modelo acepta una matriz de matrices de características (para que pueda predecir las clases de múltiples observaciones de pingüinos en una sola llamada)
# Crearemos un array con un único array de características, representando un pingüino
x_new = np.array([[50.4,15.3,224,5550]])
print ('New sample: {}'.format(x_new[0]))

# El modelo devuelve una matriz de predicciones - una para cada conjunto de características presentadas
# En nuestro caso, sólo hemos enviado un pingüino, por lo que nuestra predicción es la primera en la matriz resultante.
penguin_pred = multi_model.predict(x_new)[0]
print('Predicted class is', penguin_classes[penguin_pred])

También puedes enviar un lote de observaciones de pingüinos al modelo y obtener una predicción para cada una de ellas.